In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

Limiting the number of images per class to 400

In [2]:
root = '../input/emotion-detection-fer'
train_path = os.path.join(root, 'train')

In [3]:
files_train = list()
files_test = list()
emotion_train = list()
emotion_test = list()
for i in os.listdir(train_path):
    path = os.path.join(train_path, i)
    for j in range(400):
        if j<=(0.8*400):
            files_train.append(os.path.join(i, os.listdir(path)[j]))
            emotion_train.append(i)
        else:
            files_test.append(os.path.join(i, os.listdir(path)[j]))
            emotion_test.append(i)

In [4]:
train = pd.DataFrame([files_train, emotion_train]).T
test = pd.DataFrame([files_test, emotion_test]).T
train.columns=['filename', 'class']
test.columns=['filename', 'class']

In [5]:
test.head()

,filename,class
0,fearful/im3787.png,fearful
1,fearful/im3371.png,fearful
2,fearful/im3169.png,fearful
3,fearful/im361.png,fearful
4,fearful/im191.png,fearful


In [6]:
train.head()

,filename,class
0,fearful/im1375.png,fearful
1,fearful/im3465.png,fearful
2,fearful/im3209.png,fearful
3,fearful/im1444.png,fearful
4,fearful/im1029.png,fearful


In [7]:
datagen = ImageDataGenerator(validation_split=0.2, rescale=1./255)

train_generator = datagen.flow_from_dataframe(dataframe = train,
                                              directory = train_path,
                                              class_mode='categorical',
                                              batch_size = 32,
                                              target_size=(224, 224),
                                              subset='training',
                                              validate_filenames=False)

validation_generator = datagen.flow_from_dataframe(dataframe = train, 
                                                   directory = train_path,
                                                   class_mode='categorical',
                                                   batch_size = 32, 
                                                   target_size=(224, 224), 
                                                   subset='validation',
                                                   validate_filenames=False)


Found 1798 non-validated image filenames belonging to 7 classes.
Found 449 non-validated image filenames belonging to 7 classes.


## VGG16

In [8]:
tf.keras.backend.clear_session()

from keras.applications.vgg16 import VGG16

image_size = 224

weight_path = '../input/keras-pretrained-models/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'
vgg16_base_model = VGG16(weights=weight_path, 
                         include_top=False, 
                         input_shape=(image_size, image_size, 3))

for layer in vgg16_base_model.layers:
    layer.trainable = False

vgg16_pretrained_model = tf.keras.Sequential([
    vgg16_base_model,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(7, activation='softmax')
])
vgg16_pretrained_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
vgg16_pretrained_model.summary()

2022-11-03 17:53:34.478337: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-03 17:53:34.479301: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-03 17:53:34.489784: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-03 17:53:34.490719: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-03 17:53:34.491815: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 25088)             0         
_________________________________________________________________
dense (Dense)                (None, 256)               6422784   
_________________________________________________________________
dense_1 (Dense)              (None, 7)                 1799      
Total params: 21,139,271
Trainable params: 6,424,583
Non-trainable params: 14,714,688
_________________________________________________________________


In [9]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=2)
history_vgg16 = vgg16_pretrained_model.fit(train_generator, epochs=10, callbacks=callback)

Epoch 1/10


2022-11-03 17:53:38.268576: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-11-03 17:53:39.514368: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


57/57 [==============================] - 18s 155ms/step - loss: 2.2970 - accuracy: 0.3092
Epoch 2/10
57/57 [==============================] - 7s 126ms/step - loss: 1.1035 - accuracy: 0.5806
Epoch 3/10
57/57 [==============================] - 7s 125ms/step - loss: 0.8340 - accuracy: 0.6991
Epoch 4/10
57/57 [==============================] - 7s 122ms/step - loss: 0.6803 - accuracy: 0.7675
Epoch 5/10
57/57 [==============================] - 7s 121ms/step - loss: 0.4667 - accuracy: 0.8643
Epoch 6/10
57/57 [==============================] - 7s 120ms/step - loss: 0.3638 - accuracy: 0.9093
Epoch 7/10
57/57 [==============================] - 7s 120ms/step - loss: 0.3498 - accuracy: 0.8888
Epoch 8/10
57/57 [==============================] - 7s 123ms/step - loss: 0.3212 - accuracy: 0.9116
Epoch 9/10
57/57 [==============================] - 7s 120ms/step - loss: 0.2115 - accuracy: 0.9544
Epoch 10/10
57/57 [==============================] - 7s 121ms/step - loss: 0.1801 - accuracy: 0.9677


In [10]:
vgg16_pretrained_model.evaluate(validation_generator)

15/15 [==============================] - 3s 165ms/step - loss: 18.0122 - accuracy: 0.1960


[18.01223373413086, 0.19599108397960663]

## InceptionV3

In [11]:
tf.keras.backend.clear_session()

from keras.applications.inception_v3 import InceptionV3

weight_path = '../input/keras-pretrained-models/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
inception_base_model = InceptionV3(weights=weight_path, 
                         include_top=False, 
                         input_shape=(image_size, image_size, 3))

for layer in inception_base_model.layers:
    layer.trainable = False

inception_pretrained_model = tf.keras.Sequential([
    inception_base_model,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(7, activation='softmax')
])
inception_pretrained_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
inception_pretrained_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Functional)    (None, 5, 5, 2048)        21802784  
_________________________________________________________________
flatten (Flatten)            (None, 51200)             0         
_________________________________________________________________
dense (Dense)                (None, 256)               13107456  
_________________________________________________________________
dense_1 (Dense)              (None, 7)                 1799      
Total params: 34,912,039
Trainable params: 13,109,255
Non-trainable params: 21,802,784
_________________________________________________________________


In [12]:
history_inception = inception_pretrained_model.fit(train_generator, epochs=10, callbacks=callback)

Epoch 1/10
57/57 [==============================] - 11s 90ms/step - loss: 12.8948 - accuracy: 0.2881
Epoch 2/10
57/57 [==============================] - 5s 81ms/step - loss: 1.7410 - accuracy: 0.6151
Epoch 3/10
57/57 [==============================] - 4s 76ms/step - loss: 0.6484 - accuracy: 0.7903
Epoch 4/10
57/57 [==============================] - 4s 77ms/step - loss: 0.3422 - accuracy: 0.8776
Epoch 5/10
57/57 [==============================] - 5s 81ms/step - loss: 0.2298 - accuracy: 0.9160
Epoch 6/10
57/57 [==============================] - 5s 84ms/step - loss: 0.0740 - accuracy: 0.9822
Epoch 7/10
57/57 [==============================] - 5s 80ms/step - loss: 0.0271 - accuracy: 0.9983
Epoch 8/10
57/57 [==============================] - 4s 76ms/step - loss: 0.0156 - accuracy: 1.0000
Epoch 9/10
57/57 [==============================] - 4s 76ms/step - loss: 0.0088 - accuracy: 1.0000
Epoch 10/10
57/57 [==============================] - 4s 75ms/step - loss: 0.0070 - accuracy: 1.0000


In [13]:
inception_pretrained_model.evaluate(validation_generator)

15/15 [==============================] - 3s 114ms/step - loss: 58.4265 - accuracy: 0.1314


[58.426544189453125, 0.1314031183719635]

## Xception

In [14]:
tf.keras.backend.clear_session()

from keras.applications.xception import Xception

weight_path = '../input/keras-pretrained-models/xception_weights_tf_dim_ordering_tf_kernels_notop.h5'
xception_base_model = Xception(weights=weight_path, 
                         include_top=False, 
                         input_shape=(image_size, image_size, 3))

for layer in xception_base_model.layers:
    layer.trainable = False

xception_pretrained_model = tf.keras.Sequential([
    xception_base_model,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(7, activation='softmax')
])
xception_pretrained_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
xception_pretrained_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Functional)        (None, 7, 7, 2048)        20861480  
_________________________________________________________________
flatten (Flatten)            (None, 100352)            0         
_________________________________________________________________
dense (Dense)                (None, 256)               25690368  
_________________________________________________________________
dense_1 (Dense)              (None, 7)                 1799      
Total params: 46,553,647
Trainable params: 25,692,167
Non-trainable params: 20,861,480
_________________________________________________________________


In [15]:
history_inception_xception = xception_pretrained_model.fit(train_generator, epochs=10, callbacks=callback)

Epoch 1/10
57/57 [==============================] - 11s 138ms/step - loss: 7.7750 - accuracy: 0.2920
Epoch 2/10
57/57 [==============================] - 7s 128ms/step - loss: 1.1369 - accuracy: 0.5962
Epoch 3/10
57/57 [==============================] - 7s 130ms/step - loss: 0.6353 - accuracy: 0.7697
Epoch 4/10
57/57 [==============================] - 8s 132ms/step - loss: 0.4390 - accuracy: 0.8548
Epoch 5/10
57/57 [==============================] - 8s 132ms/step - loss: 0.2651 - accuracy: 0.9232
Epoch 6/10
57/57 [==============================] - 8s 131ms/step - loss: 0.1925 - accuracy: 0.9538
Epoch 7/10
57/57 [==============================] - 8s 131ms/step - loss: 0.1149 - accuracy: 0.9883
Epoch 8/10
57/57 [==============================] - 8s 133ms/step - loss: 0.0775 - accuracy: 0.9933
Epoch 9/10
57/57 [==============================] - 8s 133ms/step - loss: 0.0527 - accuracy: 0.9983
Epoch 10/10
57/57 [==============================] - 7s 129ms/step - loss: 0.0356 - accuracy: 1.000

In [16]:
xception_pretrained_model.evaluate(validation_generator)

15/15 [==============================] - 3s 147ms/step - loss: 23.4791 - accuracy: 0.1292


[23.47905158996582, 0.12917594611644745]

* Out of the 3 models, VGG16 performs the best on the validation data. 
* Minimum loss of 18.01 and maximum accuracy of 19.59%.